In [1]:
import torch
import torchvision
from torch.profiler import ProfilerActivity, profile, record_function

In [2]:
# ResNet50 の準備
model = torchvision.models.resnet50()
model = model.to("cuda")
model.eval()

# ダミー画像を用意
input_image = torch.ones((128, 3, 224, 224))
input_image = input_image.to("cuda")

# ウォームアップのため一度走らせる
with torch.no_grad():
    output = model(input_image)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    with record_function("model_inference"):
        model(input_image)

STAGE:2024-08-17 08:44:12 2780035:2780035 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-08-17 08:44:12 2780035:2780035 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-08-17 08:44:12 2780035:2780035 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [3]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.79%       1.627ms         6.67%      13.720ms      13.720ms       0.000us         0.00%     250.701ms     250.701ms             1  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us     204.637ms        50.04%     204.637ms     204.637ms             1  
         

In [4]:
prof.export_chrome_trace("trace.json")